# Recursive DFA-algorithm

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from transduction import (
    FSA, FST, EPSILON, PrecoverDecomp, examples, Precover,    
    display_table, format_table, HTML, colors, LazyPrecoverNFA
)

In [3]:
from transduction.dfa_decomp import RecursiveDFADecomposition, check

In [4]:
#f = examples.replace([('1', 'a'), ('2', 'b'), ('3', 'c'), ('4', 'd'), ('5', 'e')])
#target = 'abc'
f = examples.samuel_example()
target = 'cxx'

#f = examples.newspeak2()
#target = 'bad'

In [5]:
start = RecursiveDFADecomposition(f, target[:0])
check(start)
start

work {(0, '')}
✔ pass


quotient,remainder
"<?xml version=""1.0"" encoding=""UTF-8"" standalone=""no""?> <!DOCTYPE svg PUBLIC ""-//W3C//DTD SVG 1.1//EN"" ""http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd""> <!-- Generated by graphviz version 2.43.0 (0) --> <!-- Title: %3 Pages: 1 --> %3 <!-- start --> start <!-- 0 --> 0 {(0, '')} <!-- start->0 --> start->0","<?xml version=""1.0"" encoding=""UTF-8"" standalone=""no""?> <!DOCTYPE svg PUBLIC ""-//W3C//DTD SVG 1.1//EN"" ""http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd""> <!-- Generated by graphviz version 2.43.0 (0) --> <!-- Title: %3 Pages: 1 --> %3 <!-- start --> start"


In [6]:
a = (start >> target[0])
check(a)
a

work {(0, '')}
work {(1, ''),(2, 'c')}
✔ pass


quotient,remainder
"<?xml version=""1.0"" encoding=""UTF-8"" standalone=""no""?> <!DOCTYPE svg PUBLIC ""-//W3C//DTD SVG 1.1//EN"" ""http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd""> <!-- Generated by graphviz version 2.43.0 (0) --> <!-- Title: %3 Pages: 1 --> %3 <!-- start --> start <!-- 0 --> 0 {(0, '')} <!-- start->0 --> start->0 <!-- 1 --> 1 {(1, ''),(2, 'c')} <!-- 0->1 --> 0->1 a","<?xml version=""1.0"" encoding=""UTF-8"" standalone=""no""?> <!DOCTYPE svg PUBLIC ""-//W3C//DTD SVG 1.1//EN"" ""http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd""> <!-- Generated by graphviz version 2.43.0 (0) --> <!-- Title: %3 Pages: 1 --> %3 <!-- start --> start"


In [7]:
ab = (a >> target[1])
check(ab)
ab

work {(1, ''),(2, 'c')}
work {(4, 'cx')}
work {(4, 'cx'),(3, 'c')}
✔ pass


In [8]:
abc = (ab >> target[2])
check(abc)
abc

work {(4, 'cx'),(3, 'c')}
work {(4, 'cxx')}
work {(4, 'cx')}
✔ pass


In [9]:
#for x in abc.Q.language(10):
#    print(x)

### Parity

In [10]:
f = examples.parity({'a'})
s = RecursiveDFADecomposition(f, '')
check(s)
s

work {(2, '1'),(0, '')}
✔ pass


quotient,remainder
"<?xml version=""1.0"" encoding=""UTF-8"" standalone=""no""?> <!DOCTYPE svg PUBLIC ""-//W3C//DTD SVG 1.1//EN"" ""http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd""> <!-- Generated by graphviz version 2.43.0 (0) --> <!-- Title: %3 Pages: 1 --> %3 <!-- start --> start <!-- 0 --> 0 {(2, '1'),(0, '')} <!-- start->0 --> start->0","<?xml version=""1.0"" encoding=""UTF-8"" standalone=""no""?> <!DOCTYPE svg PUBLIC ""-//W3C//DTD SVG 1.1//EN"" ""http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd""> <!-- Generated by graphviz version 2.43.0 (0) --> <!-- Title: %3 Pages: 1 --> %3 <!-- start --> start"


In [11]:
s0 = s >> '0'
check(s0)
s0

work {(2, '1'),(0, '')}
work {(1, ''),(2, '0')}
work {(0, '')}
✔ pass


quotient,remainder
"<?xml version=""1.0"" encoding=""UTF-8"" standalone=""no""?> <!DOCTYPE svg PUBLIC ""-//W3C//DTD SVG 1.1//EN"" ""http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd""> <!-- Generated by graphviz version 2.43.0 (0) --> <!-- Title: %3 Pages: 1 --> %3 <!-- start --> start","<?xml version=""1.0"" encoding=""UTF-8"" standalone=""no""?> <!DOCTYPE svg PUBLIC ""-//W3C//DTD SVG 1.1//EN"" ""http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd""> <!-- Generated by graphviz version 2.43.0 (0) --> <!-- Title: %3 Pages: 1 --> %3 <!-- start --> start <!-- 0 --> 0 {(2, '1'),(0, '')} <!-- start->0 --> start->0 <!-- 2 --> 2 {(1, ''),(2, '0')} <!-- 0->2 --> 0->2 a <!-- 1 --> 1 {(0, '')} <!-- 1->2 --> 1->2 a <!-- 2->1 --> 2->1 a"


In [12]:
s1 = s >> '1'
check(s1)
s1

work {(2, '1'),(0, '')}
work {(1, '')}
work {(2, '1'),(0, '')}
✔ pass


quotient,remainder
"<?xml version=""1.0"" encoding=""UTF-8"" standalone=""no""?> <!DOCTYPE svg PUBLIC ""-//W3C//DTD SVG 1.1//EN"" ""http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd""> <!-- Generated by graphviz version 2.43.0 (0) --> <!-- Title: %3 Pages: 1 --> %3 <!-- start --> start","<?xml version=""1.0"" encoding=""UTF-8"" standalone=""no""?> <!DOCTYPE svg PUBLIC ""-//W3C//DTD SVG 1.1//EN"" ""http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd""> <!-- Generated by graphviz version 2.43.0 (0) --> <!-- Title: %3 Pages: 1 --> %3 <!-- start --> start <!-- 0 --> 0 {(2, '1'),(0, '')} <!-- start->0 --> start->0 <!-- 1 --> 1 {(1, '')} <!-- 0->1 --> 0->1 a <!-- 1->0 --> 1->0 a"


In [13]:
#for xs, ys in f.relation(5):
#    print((xs, ys))